<a href="https://colab.research.google.com/github/pv-912/predict_number/blob/master/Submission1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle
filename = 'finalized_model.sav'
model = pickle.load(open(filename, 'rb'))


In [0]:
import pandas as pd
df = pd.read_csv('gdrive/My Drive/learning/predict_number/data/train.csv')
test = pd.read_csv('gdrive/My Drive/learning/predict_number/data/test.csv')
sample = pd.read_csv('gdrive/My Drive/learning/predict_number/data/sample.csv')

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
!pip install mpld3
import mpld3
mpld3.enable_notebook()
import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
# output all lines

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
temp = test.copy()
temp.iloc[:,1] = label_encoder.fit_transform(test.iloc[:,1])
temp.head()


,ID,Tag,Reputation,Answers,Username,Views
0,366953,0,5645.0,3.0,50652,33200.0
1,71864,1,24511.0,6.0,37685,2730.0
2,141692,3,927.0,1.0,135293,21167.0
3,316833,3,21.0,6.0,166998,18528.0
4,440445,3,4475.0,10.0,53504,57240.0


In [0]:

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
X = temp[["Reputation", "Answers" , "Views"]].values.astype(np.float64)
min_max_scaler = MinMaxScaler()
min_max_scaler.fit_transform(X)
scaled_data = pd.DataFrame(min_max_scaler.fit_transform(X), columns=["Reputation" , "Answers" , "Views"])
scaled_data.head()

temp.drop(columns=["Reputation" , "Answers" , "Views"], axis=1,inplace=True)
temp = pd.concat([temp, scaled_data], axis=1)
# temp.head()

In [0]:
temp.head()

,ID,Tag,Username,Reputation,Answers,Views
0,366953,0,50652,0.005415,0.041096,0.006632
1,71864,1,37685,0.023513,0.082192,0.000544
2,141692,3,135293,0.000889,0.013699,0.004228
3,316833,3,166998,0.000020,0.082192,0.003700
4,440445,3,53504,0.004293,0.136986,0.011436


In [0]:
onehot = pd.get_dummies(temp.iloc[:,1],  prefix='tag')
# onehot.head()
temp = pd.concat([temp,onehot], axis = 1)
temp.head()

,ID,Tag,Username,Reputation,Answers,Views,tag_0,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9
0,366953,0,50652,0.005415,0.041096,0.006632,1,0,0,0,0,0,0,0,0,0
1,71864,1,37685,0.023513,0.082192,0.000544,0,1,0,0,0,0,0,0,0,0
2,141692,3,135293,0.000889,0.013699,0.004228,0,0,0,1,0,0,0,0,0,0
3,316833,3,166998,0.000020,0.082192,0.003700,0,0,0,1,0,0,0,0,0,0
4,440445,3,53504,0.004293,0.136986,0.011436,0,0,0,1,0,0,0,0,0,0


In [0]:
feature_cols = ['Reputation',  'Views']

X = temp[feature_cols]
y_pred = model.predict(X)


In [0]:
# y_pred
df.Upvotes.describe()


count    330045.000000
mean        337.505358
std        3592.441135
min           0.000000
25%           8.000000
50%          28.000000
75%         107.000000
max      615278.000000
Name: Upvotes, dtype: float64

In [0]:
maxi = y_pred*df.Upvotes.max()
maxi = pd.DataFrame( maxi, columns=[ "Upvotes" ])
maxi.head()

,Upvotes
0,221.556893
1,131.696032
2,35.882347
3,13.151701
4,299.788031


In [0]:
test.ID


In [0]:
submission = pd.concat([test['ID'], maxi['Upvotes']], axis=1, keys=['ID', 'Upvotes'])

In [0]:
submission.head()

,ID,Upvotes
0,366953,221.556893
1,71864,131.696032
2,141692,35.882347
3,316833,13.151701
4,440445,299.788031


In [0]:
submission.to_csv('/sub.csv')